In [1]:
import sys
import os
import pandas as pd
from pathlib import Path


In [2]:
# Añadir el directorio src al path de Python
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)


In [3]:
from calculadora_margen.encoder import Encoder
from calculadora_margen.cleaning import Cleaner
from calculadora_margen.cleaning.params import Parameters
from calculadora_margen.cleaning.validador import Validator
from calculadora_margen.cleaning.outliers_manager import OutliersManager

In [4]:
project_root_path = Path(src_path).parent
data_path = project_root_path / 'data'
raw_path = data_path / 'raw'
clean_path = data_path / 'clean'

ETL master_lotes

In [5]:
master_lotes = pd.read_csv(raw_path / 'costes.csv',  encoding='UTF-8', sep=';', dtype=str)

In [6]:
cleaner = Cleaner(master_lotes)
params = Parameters.master_lotes

master_lotes = (cleaner
    .keep_and_rename(params.cols_to_keep, params.rename_map)
    .drop_duplicates()
    .drop_na(params.drop_na_subset)
    .to_upper()
    .get_df()
)

In [7]:
validator = Validator(master_lotes)
master_lotes = (validator
    .validate_with_map(params.validation_map)
    .get_df()
)

In [8]:
# Creamos clave única para poder hacer merge en otros df
encoder = Encoder(master_lotes)
master_lotes = encoder.create_key(col1='articulo', col2='lote_proveedor', new_col_name='clave_merge')

In [11]:
master_lotes.sample(2)

,articulo,lote_proveedor,lote_componente,clave_merge
1499,SCRE036,041124,041124,SCRE036-041124
10774,SEM005,211122,211122,SEM005-211122


In [10]:
master_lotes.to_csv(clean_path / 'master_lotes_clean.csv', index=False)

In [12]:
# Imprimir el reporte de transformaciones
cleaner.print_report()


==================== TRANSFORMATION REPORT: unnamed ====================

Initial shape: (19326, 3)
Final shape: (19252, 3)
Total rows removed: 74
Number of transformations: 4


Detailed Steps:

1. keep_and_rename
   Description: Kept specified columns and applied renaming
   Rows before: 19326
   Rows after: 19326
   Rows affected: 0
   Details:
    - kept_columns: ['Cód. artículo', 'LOTE', 'LOTEINTERNO']
    - rename_mapping: {'Cód. artículo': 'articulo', 'LOTE': 'lote_proveedor', 'LOTEINTERNO': 'lote_componente'}
    - removed_columns: ['Cód. almacén estructura', 'DESCALM', 'Artículo', 'FECDOC', 'FECCADUC', 'UNIDADES', 'PRCMONEDA', '% descuento 1', 'TIPDOC', 'NUMDOC', 'REFERENCIA', 'Cód. proveedor', 'NOMPRO']

2. drop_duplicates
   Description: Dropped duplicate rows
   Rows before: 19326
   Rows after: 19256
   Rows affected: 70
   Details:
    - affected_columns: None

3. drop_na
   Description: Dropped rows with NA values in columns: lote_componente
   Rows before: 19326
   Rows

ETL costes

In [11]:
costes = pd.read_csv(raw_path / 'costes.csv',  encoding='UTF-8', sep=';', dtype=str)

In [12]:
cleaner = Cleaner(costes)
params = Parameters.costes

costes = (cleaner
    .drop_na(params.drop_na_subset)
    .drop_duplicates()
    .keep_and_rename(params.cols_to_keep, params.rename_map)
    .fix_numeric_format(params.cols_to_float)
    .drop_duplicates_batch(params.drop_duplicates_subset)
    .to_upper()
    .get_df()
)

In [13]:
validator = Validator(costes)

costes = (validator
    .validate_with_map(params.validation_map)
    .get_df()
)

Tamaño inicial del DataFrame: 8000

Validando columna: componente
Filas inválidas encontradas: 1

Validando columna: lote_componente
Filas inválidas encontradas: 21

Tamaño final del DataFrame: 7978


In [14]:
# Ver las filas inválidas para una columna específica
#invalid_rows = validator.get_invalid('lote_interno')
#print(invalid_rows.head(10))

In [15]:
outliers_manager = OutliersManager(costes)

costes = (outliers_manager
    .process_outliers()
    .clean_columns()
    .get_df()
)


=== RESUMEN DE OUTLIERS ===
Outliers detectados inicialmente: 71
Outliers reemplazados por la media: 66
Outliers restantes: 5

Detalle de outliers restantes:
- MAT101: 1 outliers (desviación media: 0.0%)
- MAT183: 3 outliers (desviación media: 0.0%)
- MAT265: 1 outliers (desviación media: 0.0%)


In [16]:
costes.to_csv(clean_path / 'costes_clean.csv', index=False)

In [17]:
costes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7978 entries, 0 to 7977
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   coste_componente_unitario  7978 non-null   float64
 1   lote_componente            7978 non-null   object 
 2   componente                 7978 non-null   object 
dtypes: float64(1), object(2)
memory usage: 187.1+ KB


ETL fabricaciones

In [18]:
fabricaciones = pd.read_csv(raw_path / 'fabricaciones_2025.csv',  encoding='UTF-8', sep=';', dtype=str)

In [19]:
# No queremos trabajar con lote_componente_proveedor, unimos a master_lotes para obtener lote_componente
encoder = Encoder(fabricaciones)

fabricaciones = encoder.create_key(col1='Componente', col2='Lote Componente', new_col_name='clave_merge')
fabricaciones = fabricaciones.merge(master_lotes, on="clave_merge", how="left")

In [20]:
cleaner = Cleaner(fabricaciones)
params = Parameters.fabricaciones

fabricaciones = (cleaner
    .drop_duplicates()
    .keep_and_rename(params.cols_to_keep, params.rename_map)
    .fix_numeric_format(params.cols_to_float)
    .drop_na(params.drop_na_subset)
    .to_upper()
    .get_df()
)

In [21]:
validator = Validator(fabricaciones)
fabricaciones = (validator
    .validate_with_map(params.validation_map)
    .get_df()
)

Tamaño inicial del DataFrame: 19583

Validando columna: articulo
Filas inválidas encontradas: 0

Validando columna: componente
Filas inválidas encontradas: 0

Tamaño final del DataFrame: 19583


In [22]:
fabricaciones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19583 entries, 0 to 19658
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   fecha_fabricacion    19583 non-null  object 
 1   articulo             19583 non-null  object 
 2   lote_articulo        19583 non-null  object 
 3   unidades_fabricadas  19583 non-null  float64
 4   componente           19583 non-null  object 
 5   lote_componente      19583 non-null  object 
 6   consumo_unitario     19583 non-null  float64
 7   consumo_total        19583 non-null  float64
 8   id_orden             19583 non-null  object 
dtypes: float64(3), object(6)
memory usage: 1.5+ MB


In [23]:
fabricaciones.to_csv(clean_path / 'fabricaciones_clean.csv', index=False)